In [1]:
import os
import numpy as np
from PIL import Image
import platform
from ctypes import *
import matplotlib.pyplot as plt

In [2]:
DATASET_FOLDER = "dataset"
TRAIN_SUBFOLDER = os.path.join(DATASET_FOLDER, "train")
TEST_SUBFOLDER = os.path.join(DATASET_FOLDER, "test")

TRAIN_ACTION_FOLDER= os.path.join(TRAIN_SUBFOLDER, "action")
TRAIN_COMEDY_FOLDER= os.path.join(TRAIN_SUBFOLDER, "comedie")

TEST_ACTION_FOLDER = os.path.join(TEST_SUBFOLDER, "action")
TEST_COMEDY_FOLDER= os.path.join(TEST_SUBFOLDER, "comedie")

In [6]:
def fill_x_and_y(folder, x_list, y_list, label):
    for file in os.listdir(folder):
        file_path = os.path.join(folder, file)
        try: 
            image = Image.open(file_path)
            image = image.resize((8,8))
            im_arr = np.array(image).flatten()
            x_list.append(im_arr)
            y_list.append(label)
        except:
            continue
        
def import_dataset():
    X_train = []
    Y_train = []
    X_test = []
    Y_test = []
    fill_x_and_y(TRAIN_ACTION_FOLDER, X_train, Y_train, 1.0)
    fill_x_and_y(TRAIN_COMEDY_FOLDER, X_train, Y_train, -1.0)
    fill_x_and_y(TEST_ACTION_FOLDER, X_test, Y_test, 1.0)
    fill_x_and_y(TEST_COMEDY_FOLDER, X_test, Y_test, -1.0)
    
    return (np.array(X_train) / 255.0, np.array(Y_train)), (np.array(X_test)/255.0, np.array(Y_test))

In [7]:
(X_train, Y_train), (X_test, Y_test) = import_dataset()

In [10]:
path_to_shared_library = "target/debug/librust_lib.dylib"
path_to_shared_library_windows = "target\\debug\\rust_lib.dll"

if(platform.system()=='Windows'):
    my_lib = cdll.LoadLibrary(path_to_shared_library_windows)
else : 
    my_lib = cdll.LoadLibrary(path_to_shared_library)

### Linear model

In [11]:
my_lib.create_model.argtypes = [c_int]
my_lib.create_model.restype = POINTER(c_float)

arr_size = 2
model = my_lib.create_model(arr_size)

arr_size += 1

flattened_dataset_inputs = []
for p in X_train:
    flattened_dataset_inputs.append(p[0])
    flattened_dataset_inputs.append(p[1])

flattened_inputs_len = len(flattened_dataset_inputs)
flattened_outputs_len = len(Y_train)

inputs_type = c_float * flattened_inputs_len
outputs_type = c_float * flattened_outputs_len

arr_type = type(model)

my_lib.train_rosenblatt_linear_model.argtypes = [arr_type, inputs_type, outputs_type, c_int, c_float, c_int, c_int,c_int]
my_lib.train_rosenblatt_linear_model.restype = None

inputs_native = inputs_type(*flattened_dataset_inputs)
outputs_native = outputs_type(*Y_train)
my_lib.train_rosenblatt_linear_model(model, inputs_native, outputs_native, 10000, 0.001, arr_size, flattened_inputs_len,flattened_outputs_len)

In [12]:
for p in X_test:
    p_type = c_float * len(p)
    break
my_lib.predict_linear_model_classification.argtypes = [arr_type, p_type, c_int,c_int]
my_lib.predict_linear_model_classification.restype = c_float

predicted_outputs = [my_lib.predict_linear_model_classification(model, p_type(*p),arr_size,len(p)) for p in X_test]

In [13]:
print(predicted_outputs)

[1.0, 1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, 1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]


In [9]:
my_lib.destroy_array.argtypes = [POINTER(c_float), c_int]
my_lib.destroy_array.restype = None
my_lib.destroy_array(model, arr_size)

### MLP

In [10]:
d = [2, 2, 1]
arr_size = len(d)
arr_type = c_int * arr_size
native_arr = arr_type(*d)
my_lib.create_mlp_model.argtypes = [arr_type, c_int]
my_lib.create_mlp_model.restype = c_void_p

model = my_lib.create_mlp_model(native_arr,arr_size)

flattened_dataset_inputs = []
for p in X_train:
    flattened_dataset_inputs.append(p[0])
    flattened_dataset_inputs.append(p[1])

flattened_inputs_len = len(flattened_dataset_inputs)
flattened_outputs_len = len(Y_train)

inputs_type = c_float * flattened_inputs_len
outputs_type = c_float * flattened_outputs_len

my_lib.train_classification_stochastic_backdrop_mlp_model.argtypes = [c_void_p,inputs_type,outputs_type,c_float,c_int,c_int,c_int]
my_lib.train_classification_stochastic_backdrop_mlp_model.restype = None
inputs_native = inputs_type(*flattened_dataset_inputs)
outputs_native = outputs_type(*Y_train)

my_lib.train_classification_stochastic_backdrop_mlp_model(model,inputs_native,outputs_native,0.001,100000,flattened_inputs_len,flattened_outputs_len)

In [11]:
for p in X_test:
    p_type = c_float * len(p)
    break
my_lib.predict_mlp_model_classification.argtypes = [c_void_p, p_type,c_int]
my_lib.predict_mlp_model_classification.restype = POINTER(c_float)

predicted_outputs = [my_lib.predict_mlp_model_classification(model, p_type(*p),len(p))[0] for p in X_test]

In [12]:
print(predicted_outputs)

[-0.22967755794525146, -0.31092068552970886, 0.06649662554264069, -0.43935221433639526, 0.3381742835044861, -0.3949599862098694, -0.26803603768348694, 0.6363046169281006, -0.33277884125709534, -0.29869773983955383, -0.36480680108070374, -0.5345771312713623, -0.45991238951683044, -0.26964157819747925, -0.3902912735939026]


In [13]:
my_lib.destroy_model.argtypes = [c_void_p]
my_lib.destroy_model.restype = None
my_lib.destroy_model(model)